In [1]:
import firebase_admin
from firebase_admin import firestore
from firebase_admin import credentials

# Run once
cred = credentials.Certificate('ai-writing-45138-f0d64e9114cc.json')
app = firebase_admin.initialize_app(cred)
db = firestore.client()

In [125]:
import pandas as pd

# demographic
def format_demo(dic):
    custom_order = ['q1', 'q1-other','q2', 'q3','q3','q4','q5','q6.1','q6.2','q6.3','q7','q8']
    ordered_dic = {key: dic.get(key) for key in custom_order}
    ordered_dic = {key.replace('q','D'): ordered_dic[key] for key in ordered_dic}
    return ordered_dic

# pre-study
def format_pre_study(dic):
    custom_order = ['1','2','3','4','5','6','7','8','9','10']
    new_dic = {}
    for key in custom_order:
        lst = dic.get(key)
        new_cols = {}
        for i in range(3):
            new_cols['PRE'+key+'.'+str(i+1)] = lst[i]
        new_dic.update(new_cols)
    return new_dic

def format_post_study(dic, n):
    new_dic = {}
    set1 = dic['set1']
    set2 = dic['set2']
    q_lst = ['S','P','D']
    prefix = 'POST' + str(n+1) + '.'
    for i in range(3):
        key = prefix + q_lst[i]
        idx = i+6*n
        new_dic[key] = set1[idx]
        new_dic[key+'-reason'] = set1[idx+3]
    for i in range(5):
        key = prefix + str(i+1)
        new_dic[key] = set2[n*5+i]
    return new_dic

def format_ai_task(texts, post):
    new_dic = {}
    keys = list(texts.keys())
    for i in range(2):
        num = str(i+1)
        key = keys[i]
        new_dic['AI Task '+num] = key
        new_dic['AI Text '+num] = texts.get(key)
        new_dic |= format_post_study(post,i)

    return new_dic


In [134]:
# df = pd.DataFrame()
docs = db.collection('user-input').stream()

lst = []
for doc in docs:
    content = doc.to_dict()
    new_content = {}
    new_content['ID'] = doc.id
    demo_dic = format_demo(content['demographicQuestions'])
    pre_study_dic = format_pre_study(content['preStudyAnswers'])
    new_content |= demo_dic | pre_study_dic
    new_content['Baseline Text'] = content['baselineText']
    ai_dic = format_ai_task(content['aiWritingText'],content['postStudyAnswers'])
    new_content |= ai_dic
    lst.append(new_content)
    if doc.id=='TEST2':
        # For now
        break

df = pd.DataFrame(lst)


In [135]:
df.head()

,ID,D1,D1-other,D2,D3,D4,D5,D6.1,D6.2,D6.3,...,POST2.S-reason,POST2.P,POST2.P-reason,POST2.D,POST2.D-reason,POST2.1,POST2.2,POST2.3,POST2.4,POST2.5
0,123,"Some high school, no diploma",None,1,1,1,1,2,2,2,...,second positive,2,213124,2,123,2,2,2,2,2
1,1239432,"Some high school, no diploma",None,1,1,1,1,2,2,2,...,,2,,4,,4,3,4,4,4
2,56cb314508d7300005fd312a,Master's degree,None,Bulgarian,Bulgaria,UK,Bulgaria,6,6,7,...,,3,,5,,6,6,6,7,7
3,59ada0805ea0f30001281732,Associate's degree,None,english,england,england,british,7,7,7,...,we have been working on a project so aren't st...,4,,2,,6,7,7,6,6
4,5b1dfad43522660001b4510d,Bachelor's degree,None,English,Scotland,Scotland,Scotland,7,7,7,...,I feel like I found a better balance of my own...,4,I think it's in the middle ground as we are co...,3,They might be busy so it could be a little dem...,4,5,6,6,5


In [136]:
df.to_csv('output.csv')